#### Data Summary:
* The technology firm has 3 potential hackers that perpetrated the attack.  
They are certain of the first two hackers but they aren't very sure if the third hacker was involved or not.
They have requested your help! 


In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('kmeanshack').getOrCreate()

In [3]:
df = spark.read.csv('hack_data.csv', inferSchema = True, header = True)

In [4]:
df.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|
|                   20.0|            408.5|              0|             3.57

In [5]:
df.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [6]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

In [7]:
df.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [11]:
df = df.drop('Location')

In [12]:
assembler = VectorAssembler(inputCols = df.columns, outputCol = 'features')

In [13]:
output = assembler.transform(df)

In [14]:
output.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|WPM_Typing_Speed|            features|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|           72.37|[8.0,391.09,1.0,2...|
|                   20.0|           720.99|              0|             3.04|            9.0|           69.08|[20.0,720.99,0.0,...|
|                   31.0|           356.32|              1|             3.71|            8.0|           70.58|[31.0,356.32,1.0,...|
|                    2.0|           228.08|              1|             2.48|            8.0|            70.8|[2.0,228.08,1.0,2...|
|                   20.0|            408.5|              0|             3.57

In [15]:
output.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [17]:
scaler = StandardScaler(inputCol = 'features', outputCol = 'scaledFeatures')

In [18]:
scaled_model = scaler.fit(output)

In [19]:
final_data = scaled_model.transform(output)

In [21]:
final_data.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|WPM_Typing_Speed|            features|      scaledFeatures|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|           72.37|[8.0,391.09,1.0,2...|[0.56785108466505...|
|                   20.0|           720.99|              0|             3.04|            9.0|           69.08|[20.0,720.99,0.0,...|[1.41962771166263...|
|                   31.0|           356.32|              1|             3.71|            8.0|           70.58|[31.0,356.32,1.0,...|[2.20042295307707...|
|                    2.0|           228.08|              1|             2.48|     

### Model Building

In [22]:
from pyspark.ml.clustering import KMeans

In [33]:
kmeans_k3 = KMeans(featuresCol = 'scaledFeatures', k = 3) # as per summary three hacking groups
kmeans_k2 = KMeans(featuresCol = 'scaledFeatures', k = 2)

In [34]:
model_k2 = kmeans_k2.fit(final_data)
model_k3 = kmeans_k3.fit(final_data)

In [36]:
results_k2 = model_k2.transform(final_data)
results_k3 = model_k3.transform(final_data)

In [37]:
results_k2.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [38]:
results_k3.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   88|
|         2|   79|
|         0|  167|
+----------+-----+



* Since K2 model is having even cluster K2 model is correct model, where K3 is having an uneven clusetering

In [39]:
centers = model_k2.clusterCenters()

In [40]:
print(centers)

[array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
       5.26676612]), array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
       3.28474   ])]


In [41]:
results_k2.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+--------------------+----------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|WPM_Typing_Speed|            features|      scaledFeatures|prediction|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+--------------------+----------+
|                    8.0|           391.09|              1|             2.96|            7.0|           72.37|[8.0,391.09,1.0,2...|[0.56785108466505...|         0|
|                   20.0|           720.99|              0|             3.04|            9.0|           69.08|[20.0,720.99,0.0,...|[1.41962771166263...|         0|
|                   31.0|           356.32|              1|             3.71|            8.0|           70.58|[31.0,356.32,1.0,...|[2.20042295307707...|         0|
|               

### Model Evaluation

In [31]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [43]:
wssse = model_k2.summary.trainingCost
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(results_k2)

print(f"Within Set Sum of Squared Errors: {wssse}")
print(f"Silhouette with squared euclidean distance: {silhouette}")

Within Set Sum of Squared Errors: 601.7707512676691
Silhouette with squared euclidean distance: 0.6683623593283755
